In [1]:
%load_ext autoreload
%autoreload 2

import json
import numpy as np
from pydrake.geometry import StartMeshcat

from iiwa_batter import PACKAGE_ROOT, CONTACT_TIMESTEP
from iiwa_batter.swing_optimization.instantaneous_swing import run_instantaneous_swing
from iiwa_batter.robot_constraints.get_joint_constraints import JOINT_CONSTRAINTS

Loaded joint constraints for KUKA robots


In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [ ]:
# Load plate ball state and load it into numpy arrays
with open(f"{PACKAGE_ROOT}/sandbox/ball_plate_state_90mph.json", "r") as f:
    ball_state = json.load(f)

ball_plate_position = np.zeros(7)
ball_plate_velocity = np.zeros(6)
for key, value in ball_state.items():
    if key == "position":
        for i, val in enumerate(value.items()):
            ball_plate_position[i] = val[1]
    elif key == "velocity":
        for i, val in enumerate(value.items()):
            ball_plate_velocity[i] = val[1]

print(f"Plate ball position: {ball_plate_position[4:]}")
print(f"Plate ball velocity: {ball_plate_velocity[3:]}")
plate_ball_state = (ball_plate_position, ball_plate_velocity)

# Our optimization is over the following parameters:
# Initial iiwa position, which is a 8x1 vector (base rotation and all the joint rotations)
# Initial iiwa velocity, which is a 7x1 vector (all the joint velocities)
# iiwaa torque is also a 7x1 vector, but we don't optimize over it. Simply set as maximum in direction of velocity



plate_iiwa_position = np.array([0, 0, 0, 0, 0, 0, 0, 0])
plate_iiwa_velocity = np.array([0, 0, 10, 0, 0, 0, 0])


Plate ball position: [0.33528    0.         0.63797414]
Plate ball velocity: [-40.2336       0.          -4.78089136]


In [9]:
plate_iiwa_position = np.array([0, 1.5, -1.5, 1.6, -0.4, -1.7, 1.6, 0])
plate_iiwa_velocity = np.array([0, 0, 0, 0, 0, 0, 0])
run_instantaneous_swing(meshcat, plate_iiwa_position, plate_iiwa_velocity, plate_ball_state, CONTACT_TIMESTEP)

-2.1034870466506552

|   iter    |  target   |  joint1   |  joint2   |  joint3   |  joint4   |  joint5   |  joint6   |  joint7   |
-------------------------------------------------------------------------------------------------------------
| 2         | -0.4389   | -0.4582   | -0.3063   | 0.1355    | -0.2115   | 0.8405    | -1.393    | 1.782     |
| 4         | 0.3438    | 1.389     | -0.5536   | 0.6713    | 0.9854    | 1.791     | -1.955    | -2.172    |
| 9         | 0.7203    | 0.2199    | -1.048    | 0.3117    | 0.523     | -1.805    | -0.405    | 0.9161    |
| 11        | 0.9602    | 1.197     | -1.076    | -1.259    | 0.8047    | -0.4643   | -1.577    | 2.015     |
| 16        | 1.192     | 0.366     | -1.437    | 1.499     | 0.4998    | 2.257     | -1.544    | -1.71     |
| 36        | 1.277     | 0.3324    | -1.324    | -0.2786   | 0.4688    | 1.9       | -2.354    | 2.247     |
| 101       | 2.384     | 0.4438    | -1.484    | 1.745     | 0.5494    | 0.3419    | -2.356    | 0.6893    |
| 103     